<a href="https://colab.research.google.com/github/ak-org/notebooks/blob/master/transferlearning_resnet50_cifar10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 80kB/s 
     |████████████████████████████████| 4.3MB 42.9MB/s 
     |████████████████████████████████| 501kB 48.4MB/s 


In [0]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50     import ResNet50
from tensorflow.keras.applications.vgg16        import VGG16
from tensorflow.keras.applications.vgg19        import VGG19
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications.inception_v3 import preprocess_input as incv3_preprocess_input
from tensorflow.keras.applications.resnet50     import preprocess_input as resnet50_preprocess_input
from tensorflow.keras.applications.vgg16        import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.applications.vgg19        import preprocess_input as vgg19_preprocess_input

import numpy as np

def custom_resnet_cifar(img_d = 128, img_c= 3):
    conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_d, img_d, img_c))
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])
    print(model.summary())
    return model 



In [2]:
print(tf.__version__)

2.0.0-rc1


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

onehot_y_train = to_categorical(y_train, 10)
onehot_y_test = to_categorical(y_test, 10)

print( X_train.shape, y_train.shape,  X_test.shape, y_test.shape )

upscaled_x_train = tf.keras.backend.resize_images(X_train, 
                                                  height_factor = 4, 
                                                  width_factor = 4, 
                                                  data_format = "channels_last")

## preprocess input
print("Pre processing inputs")
upscaled_x_train = resnet50_preprocess_input(upscaled_x_train)
print(upscaled_x_train.shape)
print("Building custom model")
custom_model = custom_resnet_cifar()
print("Training")
history = custom_model.fit(upscaled_x_train, onehot_y_train, 
                           epochs=5, batch_size=20,
                           shuffle = True, 
                           validation_split=0.2)
custom_model.save('cifar_resnet_model_v2.h5')


170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
Pre processing inputs
(50000, 128, 128, 3)
Building custom model
94773248/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 4, 4, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32768)             131072    
_________________________________________________________________
dense (Dense)                (None, 128)               4194432   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0       

In [4]:
upscaled_x_test = tf.keras.backend.resize_images(X_test, 
                                                  height_factor = 4, 
                                                  width_factor = 4, 
                                                  data_format = "channels_last")

## preprocess input
print("Pre processing test images.")
upscaled_x_test = resnet50_preprocess_input(upscaled_x_test)
print(upscaled_x_test.shape)

Pre processing test images.
(10000, 128, 128, 3)


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
upscaled_x_test = np.array(upscaled_x_test)

y_pred = custom_model.predict_classes(upscaled_x_test.astype(float), batch_size= 20)



In [11]:

matrix = confusion_matrix(y_test[:,0], y_pred)
print("Confusion Matrix")
print(matrix)
acc = accuracy_score(y_test[:,0], y_pred)
print("Accuracy : {}".format(acc * 100.0))

Confusion Matrix
[[954   3   6   2   0   0   3   2  22   8]
 [  5 958   0   0   0   0   0   2   2  33]
 [ 17   0 924  19  21   3   5   7   4   0]
 [  5   1  18 888  13  49  16   5   3   2]
 [  4   2  22  27 904   6   6  23   6   0]
 [  0   0  11 111  16 837   6  16   3   0]
 [  5   0  14  16   3   4 951   2   5   0]
 [  5   0   6  12  16  14   0 946   0   1]
 [ 25   8   1   0   0   0   1   0 963   2]
 [  6  25   1   2   0   0   0   1  10 955]]
Accuracy : 92.80000000000001
